Pretrain Models

In [15]:

#%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sys, os
from fnmatch import fnmatch
from keras.models import Sequential 
from keras.layers import Activation, Dense, Flatten,Conv1D, MaxPooling1D,Dropout,GlobalAveragePooling1D

from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate
import glob

In [1]:
# to get all .pkl files in a base directory. It will go to every sub directory find the .pkl file

signals = [] # it will store all EDA signals of all subjects from "chest" device
root = "Dataset/WESAD/untitled folder/"
pattern = "*.pkl"
signal_name='EDA'

file_paths = glob.glob(root+pattern)
subs=[]
for path in  file_paths:
    print(path)
    with open(path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')
    eda_signal= data['signal']['chest'][signal_name][:,0]
    label=data['label']
    numpy_data1=np.array([eda_signal,label])
    numpy_data1=numpy_data1.T
    df = pd.DataFrame(data=numpy_data1, columns=[signal_name,"Label"]) 
    df= df[df["Label"]==1].reset_index(drop=True)
    signals.append(df[signal_name].values)
            

In [17]:
subjects=[x.split('/')[-1].split('.')[0] for x in file_paths]
import random
def delete_random_elems(input_list, n):
    to_delete = set(random.sample(range(len(input_list)), n))
    return [x for i,x in enumerate(input_list) if not i in to_delete]


signal_same_length = []

for signal in signals:
    if len(signal)==min([len(x) for x in signals]):
        signal_same_length.append(signal)
    else:
        signal_same_length.append(np.array(delete_random_elems(signal, len(signal)-min([len(x) for x in signals]))))


In [19]:
def build_data_with_forecasting(signal,window,output_length, overlap):
    X=[]
    y=[]
    start=0
    while (True):
        x1=signal[start:start+window]
        output=signal[start+window:start+window+output_length] # 10 points
        if (len(output)!=output_length):
            break
        X.append(x1)
        y.append(output)
        start=start+overlap
    return X,y

In [ ]:
import math
def viz(history,number,subject,signal_name,data_point):
    plt.plot(history.history['mean_absolute_error'])
    plt.plot(history.history['val_mean_absolute_error'])
    plt.title('Mean Absolute Error')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("Results/Visualization/PretrainModelsWithForecastingAndRandomDatapoints/"+signal_name+"/"+subject+"/10Seconds_"+str(number)+"_"+str(data_point)+"_MAE.png",dpi=500)
    plt.show()


    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("Results/Visualization/PretrainModelsWithForecastingAndRandomDatapoints/"+signal_name+"/"+subject+"/10Seconds_"+str(number)+"_"+str(data_point)+"_Loss.png",dpi=500)
    plt.show()


    
def evaluate_model_for_bar(trainX,trainy,testX,testy,window_size,output,number,subject,signal_name,data_point):
    epoch=60
#     trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.33, random_state=42)
    trainX,trainy,testX,testy = np.array(trainX), np.array(trainy), np.array(testX), np.array(testy)
    
    model = tf.keras.models.Sequential()
    model.add(Conv1D(40, 4, activation='leaky_relu',padding="same", input_shape=(trainX.shape[1],1)))

    model.add(Conv1D(30, 2,padding="same", activation='leaky_relu'))

    model.add(Conv1D(18, 4,padding="same", activation='leaky_relu'))

    model.add(Conv1D(30, 2,padding="same", activation='leaky_relu'))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(70, activation='leaky_relu'))
    model.add(Dense(30, activation='leaky_relu'))
    model.add(Dense(output, activation='linear'))
    model.compile(loss="mean_squared_error",optimizer='adam', metrics=['mean_squared_error'])

    model.fit(trainX, trainy, validation_split=0.1,epochs=epoch, batch_size=1000, verbose=0)
    # evaluate model
    _,rmse = model.evaluate(testX, testy,batch_size=1000, verbose=0)
    rmse = math.sqrt(rmse)
    
    #save models
    model.save("Models/PretrainModelsWithForecastingAndRandomDatapoints/"+signal_name+"/"+subject+"/RMSE_V2_10Seconds_"+str(number)+"_"+str(data_point)+"_model.h5")

    
    df=pd.DataFrame({
                    'Subject':[subject],
                    'Data Point':[data_point],
                    'Model No':[number],
                     'Window Size': [window_size],
                     'Output':[output],
        
                    'RMSE':[rmse],
        
                    'Signal':[signal_name]
    })
    print(rmse)
    return df


In [2]:
import random

data_points=[7000]

signal='EDA'

window_size=[7000]

overlap=100
subject_no=1
for subject in subjects[subject_no-1:]:
    evaluation_list=[]
    z=1
    for data_point in data_points:
        for win in window_size:
            for output_length in [40]:
                print(subjects[subject_no-1],end=",")
                print(data_point)
                X,y=build_data_with_forecasting(signal_same_length[subject_no-1],win,output_length,overlap)
                testX=X[910:]
                testy=y[910:]
                randomlist = random.sample(range(0, 7000), data_point) 
                trainX= [X[i] for i in randomlist] 
                trainy=[y[i] for i in randomlist]
                eval_model= evaluate_model_for_bar(trainX,trainy,testX,testy,win,output_length,z,subject,signal,data_point)
                z=z+1
                evaluation_list.append(eval_model)
    subject_no=subject_no+1
    final_df= pd.concat(evaluation_list)
    final_df.to_csv(r"Models/PretrainModelsWithForecastingAndRandomDatapoints/"+signal+"/"+subject+"/RMSE_V2_10Seconds.csv",index=False)
    